In [1]:
import pandas as pd
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sandp_df = table[0]
sandp_df.to_csv('S&P500-Info.csv')
sandp_df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

#https://medium.com/wealthy-bytes/5-lines-of-python-to-automate-getting-the-s-p-500-95a632e5e567

In [2]:
sandp_df

#so the symbol is the same as the corresponding stock ticker. It will be used for parsing news results that reference the company that made the headlines.

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
5,ATVI,Activision Blizzard,reports,Communication Services,Interactive Home Entertainment,"Santa Monica, California",2015-08-31,718877,2008
6,ADBE,Adobe Inc.,reports,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982
7,AMD,Advanced Micro Devices Inc,reports,Information Technology,Semiconductors,"Santa Clara, California",2017-03-20,2488,1969
8,AAP,Advance Auto Parts,reports,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932
9,AES,AES Corp,reports,Utilities,Independent Power Producers & Energy Traders,"Arlington, Virginia",1998-10-02,874761,1981


In [3]:
# Import libraries
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

finwiz_url = 'https://finviz.com/quote.ashx?t='

# Example code

Taken from https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638

In [4]:
news_tables = {}
tickers = sandp_df['Symbol'].iloc[:10] #locates a selection based on the position index. In this case I am just selecting the first 10 rows under the column symbol

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table
    
tickers

0     MMM
1     ABT
2    ABBV
3    ABMD
4     ACN
5    ATVI
6    ADBE
7     AMD
8     AAP
9     AES
Name: Symbol, dtype: object

In [5]:
# Read one single day of headlines for 'MMM' 
mmm = news_tables['MMM']
# Get all the table rows tagged in HTML with <tr> into 'amzn_tr'
mmm_tr = mmm.findAll('tr')

for i, table_row in enumerate(mmm_tr):
    # Read the text of the element 'a' into 'link_text'
    a_text = table_row.a.text
    # Read the text of the element 'td' into 'data_text'
    td_text = table_row.td.text
    # Print the contents of 'link_text' and 'data_text' 
    print(a_text)
    print(td_text)
    # Exit after printing 4 rows of data
    if i == 3:
        break

30 Dividend Kings of 2021 (Part III)
Feb-06-21 09:12AM  
3M, Yum! Brands, and Other Companies That Raised Their Dividends This Week
07:30AM  
3 Stocks to Help You Join the $100 Billion Dollar Club
07:15AM  
Income Investors Should Know That 3M Company (NYSE:MMM) Goes Ex-Dividend Soon
02:53AM  


In [17]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text() 
        # splite text in the td tag into a list 
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        # else load 'date' as the 1st element and 'time' as the second    
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'  
        ticker = file_name.split('_')[0]
        
        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])
        
# Set column names
columns = ['ticker', 'date', 'time', 'headline']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_news_updated = pd.DataFrame(parsed_news, columns=columns)
parsed_news_updated.head(20)

,ticker,date,time,headline
0,MMM,Feb-06-21,09:12AM,30 Dividend Kings of 2021 (Part III)
1,MMM,Feb-06-21,07:30AM,"3M, Yum! Brands, and Other Companies That Rais..."
2,MMM,Feb-06-21,07:15AM,3 Stocks to Help You Join the $100 Billion Dol...
3,MMM,Feb-06-21,02:53AM,Income Investors Should Know That 3M Company (...
4,MMM,Feb-04-21,10:05AM,3M and Discovery Education Open Call for Entri...
5,MMM,Feb-04-21,08:25AM,AIRO.LIFE secures Trademark in the United Stat...
6,MMM,Feb-02-21,07:53PM,Dow Inc CEO Jim Fitterling elected to 3M's board
7,MMM,Feb-02-21,07:15PM,3M Board Declares Increase to First Quarter 20...
8,MMM,Feb-02-21,06:59PM,Jim Fitterling Elected to 3M Board of Directors
9,MMM,Feb-02-21,11:43AM,"Even Before WallStreetBets, Shorting GameStop ..."


In [15]:
#need to tokenize each words within the headlines to improve the sentiment score.

import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def regex(x):
    special_chars_p = "[.®'&$’\"\-()#@!?/:]"
    s1 = re.sub(special_chars_p, '',x)  
    return(s1)

parsed_news_updated = parsed_news_updated.apply(regex)

stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.lower().split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

parsed_news_updated=parsed_news_updated.apply(stem_sentences)
parsed_news_updated.head()
    
stop=stopwords.words('english')

parsed_news_updated.apply(lambda x: [item for item in x if item not in stop])

parsed_news_updated = parsed_news_updated.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)])) 

TypeError: ('expected string or bytes-like object', 'occurred at index ticker')

In [16]:
# NLTK VADER for sentiment analysis
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

# Iterate through the headlines and get the polarity scores using vader
scores = parsed_news_updated['headline'].apply(vader.polarity_scores).tolist()

# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)

# Join the DataFrames of the news and the list of dicts
parsed_and_scored_news = parsed_news_updated.join(scores_df, rsuffix='_right')

# Convert the date column from string to datetime
parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head(10)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,ticker,date,time,headline,compound,neg,neu,pos
0,MMM,2021-02-06,09:12AM,30 Dividend Kings of 2021 (Part III),0.0000,0.0,1.000,0.000
1,MMM,2021-02-06,07:30AM,"3M, Yum! Brands, and Other Companies That Rais...",0.0000,0.0,1.000,0.000
2,MMM,2021-02-06,07:15AM,3 Stocks to Help You Join the $100 Billion Dol...,0.5994,0.0,0.620,0.380
3,MMM,2021-02-06,02:53AM,Income Investors Should Know That 3M Company (...,0.0000,0.0,1.000,0.000
4,MMM,2021-02-04,10:05AM,3M and Discovery Education Open Call for Entri...,0.2023,0.0,0.878,0.122
5,MMM,2021-02-04,08:25AM,AIRO.LIFE secures Trademark in the United Stat...,0.6249,0.0,0.683,0.317
6,MMM,2021-02-02,07:53PM,Dow Inc CEO Jim Fitterling elected to 3M's board,0.0000,0.0,1.000,0.000
7,MMM,2021-02-02,07:15PM,3M Board Declares Increase to First Quarter 20...,0.3182,0.0,0.777,0.223
8,MMM,2021-02-02,06:59PM,Jim Fitterling Elected to 3M Board of Directors,0.0000,0.0,1.000,0.000
9,MMM,2021-02-02,11:43AM,"Even Before WallStreetBets, Shorting GameStop ...",0.0000,0.0,1.000,0.000
